In [179]:
import os
import torch
import numpy as np
import pandas as pd

from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, utils
from torch.nn.utils.parametrizations import weight_norm
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [180]:
train_dataset_cifar = datasets.CIFAR10('data', train=True, download=True,
transform=transforms.Compose([
    transforms.ToTensor()
]))

test_dataset_cifar = datasets.CIFAR10('data', train=False, download=True,
transform=transforms.Compose([
    transforms.ToTensor()
]))

Files already downloaded and verified
Files already downloaded and verified


In [194]:
# Inspired by https://github.com/bjlkeng/sandbox/blob/master/realnvp/pytorch-realnvp-cifar10.ipynb
check_mask = {}
check_mask_device = {}
# Partitions the input image into two sets of variables
def partition_mask(shape, to_device=True):
    global check_mask, check_mask_device
    if shape not in check_mask:
        check_mask[shape] = 1 - np.indices(shape).sum(axis=0) % 2 
        check_mask[shape] = torch.Tensor(check_mask[shape])
        
    if to_device and shape not in check_mask_device:
        check_mask_device[shape] = check_mask[shape].to(device)
        
    return check_mask_device[shape] if to_device else check_mask[shape]


chan_mask = {}
chan_mask_device = {}
# Segregrates the channels into two groups, which transformations are independently applied
def channel_mask(shape, to_device=True):
    global chan_mask, chan_mask_device
    if shape not in chan_mask:
        chan_mask[shape] = torch.cat([torch.zeros((shape[0] // 2, shape[1], shape[2])),
                                      torch.ones((shape[0] // 2, shape[1], shape[2])),],
                                      dim=0)
        assert chan_mask[shape].shape == shape, (chan_mask[shape].shape, shape)
        
    if to_device and shape not in chan_mask_device:
        chan_mask_device[shape] = chan_mask[shape].to(device)
        
    return chan_mask_device[shape] if to_device else chan_mask[shape]

### Class for Normalizing Flows with Real NVP

In [195]:
class ConvBlock(nn.Module):
    def __init__(self, in_planes, out_planes) -> None:
        super().__init__()
        self.conv1 = weight_norm(nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=1, padding=1, dilation=1, bias=False))
        self.bn1 = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = weight_norm(nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, dilation=1, bias=False))
        self.bn2 = nn.BatchNorm2d(out_planes)

        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += x
        return self.relu(out)

In [202]:
class NormalizingFlow_NVP(nn.Module):
    def __init__(self, num_coupling=18, num_final_coupling=4, planes=64, shape=(3,32,32)):
        super(NormalizingFlow_NVP, self).__init__()
        self.num_coupling = num_coupling # Number of initial coupling layers
        self.num_final_coupling = num_final_coupling # Number of final coupling layers
        self.shape = shape # Shape of the input image
        
        self.planes = planes # Number of output planes in the convolutional layers
        self.s = nn.ModuleList() # Scaling functions for each coupling layer
        self.t = nn.ModuleList() # Translation functions for each coupling layer 
        self.norms = nn.ModuleList() # List of batch normalization layers 
        
        # Learnable scalar scaling parameters for outputs of s and t
        self.s_scale = nn.ParameterList()
        self.t_scale = nn.ParameterList()
        self.t_bias = nn.ParameterList()
        self.shapes = []

        # Change shape and planes to increase model's capacity
        for i in range(num_coupling):
            self.append_transformations(shape, planes)
            if i % 6 == 2:
                shape = (4 * shape[0], shape[1] // 2, shape[2] // 2)
            if i % 6 == 5:
                # Factoring out half the channels
                shape = (shape[0] // 2, shape[1], shape[2])
                planes = 2 * planes
       
        # Setup final coupling layers with possibly different configurations
        for i in range(num_final_coupling):
            self.append_transformations(shape, planes)
           
        self.validation = False # Flag to indicate if the model is in validation mode
    
    def conv_stack(self, shape:int, planes: int):
        # A common stack of convolutional blocks used in s and t functions
        return nn.Sequential(
            weight_norm(nn.Conv2d(shape[0], planes, kernel_size=3, stride=1, padding=1, groups=1, bias=False, dilation=1)),
            nn.InstanceNorm2d(planes),
            ConvBlock(planes, planes),
            ConvBlock(planes, planes),
            ConvBlock(planes, planes),
            ConvBlock(planes, planes),
            weight_norm(nn.Conv2d(planes, shape[0], kernel_size=3, stride=1, padding=1, groups=1, bias=False, dilation=1)),
            nn.InstanceNorm2d(planes),
        )
    
    def append_transformations(self, shape: int, planes: int):
        # Append scaling and translation functions for each coupling layer
        self.s.append(self.conv_stack(shape, planes))
        self.t.append(self.conv_stack(shape, planes))
        self.s_scale.append(torch.nn.Parameter(torch.zeros(shape), requires_grad=True))
        self.t_scale.append(torch.nn.Parameter(torch.zeros(shape), requires_grad=True))
        self.t_bias.append(torch.nn.Parameter(torch.zeros(shape), requires_grad=True)) 
        self.norms.append(nn.BatchNorm2d(shape[0]))
        self.shapes.append(shape)

    def validate(self): 
        # Set the model to validation mode
        self.eval()
        self.validation = True

    def train(self, mode=True): 
        # Set the model to training mode
        nn.Module.train(self, mode)
        self.validation = False

    def get_binary_mask(self, shape: int, i: int):
        # Apply mask to manage which parts of the data are transformed
        if i in range(self.num_coupling):
            binary_mask = partition_mask(shape) if i % 6 < 3 else channel_mask(shape)
        elif i in range(self.num_coupling, self.num_coupling + self.num_final_coupling):
            binary_mask = partition_mask(shape)  
        else:
            raise ValueError("Invalid coupling layer index.")
        return  binary_mask if i % 2 == 0 else (1 - binary_mask)


    def set_transformation(self, is_reverse, b: torch.Tensor, input_tensor: torch.Tensor, layer_i: int):
        # Compute scaling and translation functions for each coupling layer
        t = (self.t_scale[layer_i]) * self.t[layer_i](b * input_tensor) + (self.t_bias[layer_i])
        s = (self.s_scale[layer_i]) * torch.tanh(self.s[layer_i](b * input_tensor))
        # Apply transformation
        if not is_reverse:
            x = input_tensor 
            return b * x + (1 - b) * (x * torch.exp(s) + t), s
        else:
            y = input_tensor 
            return b * y + (1 - b) * ((y - t) * torch.exp(-s))

    def forward(self, x: torch.Tensor):
        # Forward pass through the normalizing flow model
        if self.training or self.validation:
            # List to collect scaling outputs  / batch normalizaiton layers / outputs from each coupling layer
            s_vals = [] 
            norm_vals = [] 
            y_vals = [] 

            # Process through each coupling layer
            for i in range(self.num_coupling):
                shape = self.shapes[i]
                b_mask = self.get_binary_mask(shape, i)
                print(i)
                print("b_mask", b_mask.shape)
                print("x", x.shape)
                print("self.s_scale[i]", self.s_scale[i].shape)
                print("self.t_scale[i]", self.t_scale[i].shape)
                print("bmask * x", (b_mask * x).shape)
                print("self.t_bias[i]", self.t_bias[i].shape)
                print("self.s[i](b_mask*x)", self.s[i](b_mask * x).shape)
                print("self.t[i](b_mask*x)", self.t[i](b_mask * x).shape)
                y, s = self.set_transformation(False, b_mask, x, i)
                s_vals.append(torch.flatten((1 - b_mask) * s))

                # Apply batch normalization if available and collect outputs
                if self.norms[i] is not None:
                    y, norm_loss = self.norms[i](y)
                    norm_vals.append(norm_loss)

                # Update shape for pixel operations
                if i % 6 == 2:
                    y = torch.nn.functional.pixel_unshuffle(y, 2)

                # Manage channel factors for dimension management
                if i % 6 == 5:
                    factor_channels = y.shape[1] // 2
                    y_vals.append(torch.flatten(y[:, factor_channels:, :, :], 1))
                    y = y[:, :factor_channels, :, :]

                x = y

            # Apply final coupling layers
            for i in range(self.num_coupling, self.num_coupling + self.num_final_coupling):
                shape = self.shapes[i]
                b_mask = self.get_binary_mask(shape, i)
                y, s = self.set_transformation(False, b_mask, x, i)
                s_vals.append(torch.flatten((1 - b_mask) * s))

                if self.norms[i] is not None:
                    y, norm_loss = self.norms[i](y)
                    norm_vals.append(norm_loss)

                x = y

            y_vals.append(torch.flatten(y, 1))

            # Aggregate outputs and various losses for determinant computation
            return (torch.flatten(torch.cat(y_vals, 1), 1),
                    torch.cat(s_vals), 
                    torch.cat([torch.flatten(v) for v in norm_vals]) if len(norm_vals) > 0 else torch.zeros(1),
                    torch.cat([torch.flatten(s) for s in self.s_scale]))
        else:
            # Reverse transformation for data generation
            y = x
            y_remaining = y

            layer_vars = np.prod(self.shapes[-1])
            y = torch.reshape(y_remaining[:, -layer_vars:], (-1,) + self.shapes[-1])
            y_remaining = y_remaining[:, :-layer_vars]

            # Reversed operations for final checkerboard and coupling layers
            for i in reversed(range(self.num_coupling, self.num_coupling + self.num_final_coupling)):
                if self.norms[i] is not None:
                    y, _ = self.norms[i](y)
              
                shape = self.shapes[i]
                b_mask = self.get_binary_mask(shape, i)
                x = self.set_transformation(True, b_mask, y, i)
                y = x           

            # Prepate for multi-scale operations
            layer_vars = np.prod(shape)
            y = torch.cat((y, torch.reshape(y_remaining[:, -layer_vars:], (-1,) + shape)), 1)
            y_remaining = y_remaining[:, :-layer_vars]

            # Multi-scale coupling layers (Reverse transformations for earlier layers)
            for i in reversed(range(self.num_coupling)):
                shape = self.shapes[i]
                b_mask = self.get_binary_mask(shape, i)

                if self.norms[i] is not None:
                    y, _ = self.norms[i](y)

                x = self.set_transformation(True, b_mask, y, i)

                if i % 6 == 3:
                    x = torch.nn.functional.pixel_shuffle(x, 2)

                y = x

                if i > 0 and i % 6 == 0:
                    layer_vars = np.prod(shape)
                    y = torch.cat((y, torch.reshape(y_remaining[:, -layer_vars:], (-1,) + shape)), 1)
                    y_remaining = y_remaining[:, :-layer_vars]

            assert np.prod(y_remaining.shape) == 0
            return x

In [197]:
def get_loss(y, s, norms, batch_size, scale):
    # loss = priori gaussiana + determinante + batch_norm_scalers
    # loss = - (-0.5 * log(2 pi) - 0.5 * y**2 + s1 + s2 + ... + batch_norm_scalers)
    log_prior = -torch.sum(0.5 * torch.log(2 * torch.tensor(np.pi)) + 0.5 * y ** 2) # priori gaussiana
    log_determinant = torch.sum(s) 
    batch_norms = torch.sum(norms)
    l2_penalty = 0.0001 * torch.sum(scale ** 2)
    loss = -(log_prior + log_determinant + batch_norms) + l2_penalty
    return torch.div(loss, batch_size)

In [199]:
num_pixels = 32*32*3
def pre_processing(x):
    x = x * 255. # 0-1 -> 0-255
    x += torch.rand_like(x) # Add random uniform [0, 1] noise 
    x = torch.logit(0.05 + 0.95 * x / 256) # Apply transform to deal with boundary effects
    return x

def train_model(model, optimizer, batch_size, report_iters=10):
    train_loader = DataLoader(train_dataset_cifar, batch_size=batch_size, shuffle=True)
    avg_loss, size = 0, len(train_loader.dataset)
    for batch, (X, _) in enumerate(train_loader):
        X = pre_processing(X)
        X = X.to(device)

        # Compute loss and gradients
        y, s, norms, scale = model(X)
        loss = get_loss(y, s, norms, batch_size, scale)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Report progress
        loss += loss.item()
        if (batch + 1) % report_iters == 0:
            print(f'Batch {batch + 1}/{size // batch_size}; Loss: {loss}')

    avg_loss = loss / (batch + 1)
    #print(f"Avg. Loss: {avg_loss:.2f}")
    return avg_loss

def test_model(model, batch_size):
    test_loader = DataLoader(test_dataset_cifar, batch_size, shuffle=True)
    test_loss, size = 0, len(test_loader.dataset)

    with torch.no_grad():
        model.validate()
        for X, _ in test_loader:
            X = pre_processing(X)
            X = X.to(device)

            # Compute loss and gradients
            y, s, norms, scale = model(X)
            loss = get_loss(y, s, norms, batch_size, scale)

            test_loss += loss.item()
            #test_loss -= torch.sum(log_preprocessing_gradient(X)) / batch_size
        model.train()
    
    test_loss /= size
    #print(f"Test Error: \n Avg. Loss: {test_loss:.2f}")
    return test_loss

In [200]:
# Hyperparameters
learning_rate = 0.0005
batch_size = 20
epochs = 20
model = NormalizingFlow_NVP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
PATH = 'models/'

In [201]:
from tqdm import tqdm
# Training
best_val_loss, best_path = float('inf'), None
for t in tqdm(range(epochs), desc="Epochs"):
    train_loss = train_model(model, optimizer, batch_size)
    val_loss = test_model(model, batch_size)

    if val_loss < best_val_loss:
        best_val_loss, best_path = val_loss, PATH + f'cifar10_model_{t + 1}.pth'
        torch.save(model.state_dict(), os.path.join(best_path))
    scheduler.step()

print(f"Done! \n Best Path: {PATH + f'cifar10_model_{t + 1}.pth'} \n Best Loss: {best_val_loss:.2f}")

Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

0
b_mask torch.Size([3, 32, 32])
x torch.Size([20, 3, 32, 32])
self.s_scale[i] torch.Size([3, 32, 32])
self.t_scale[i] torch.Size([3, 32, 32])
bmask * x torch.Size([20, 3, 32, 32])
self.t_bias[i] torch.Size([3, 32, 32])


C:\Users\iaram\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\instancenorm.py:80: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "


self.s[i](b_mask*x) torch.Size([20, 3, 32, 32])
self.t[i](b_mask*x) torch.Size([20, 3, 32, 32])


Epochs:   0%|          | 0/20 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 2)